# The following code is responsible for converting MIDI files into text notation.

The whole code is based on the following model: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/ 

# I found a way to use the original python library!

The original code was developed using Python2 and the Python-midi library, which is not available for Python3.

Basically i found a Python3 compatible version of the original library.


Reference here: https://github.com/jameswenzel/mydy/blob/master/src/Containers.py
https://github.com/jameswenzel/Fractal-Midi/blob/master/script.py
https://github.com/vishnubob/python-midi

## Opening MIDI file
Basically the next cell opens and reads a dummy MIDI file written by me.

I'm also setting the  ```resolution ``` parameter to 480. This parameter is equivalent to **PPQ** in MIDI files.

**PPQ** (*Pulse per Quarter Note*) is a fixed value which sets the number of pulses contained in a quarter note, it's like a "sampling" frequency.

Each quarter note (no matter what is the original speed of the song) will contain 480 pulses. Then these pulses are converted into actual playback using the **Tempo** information of the MIDI file (obviously a quarter note at 100 BPM is slower than a quarter note at 130 BPM)

In [27]:
from mydy import Events, FileIO, Containers, Constants
test=FileIO.read_midifile('sad but true.mid') #returns a Pattern with the MIDI file information (resolution ecc...), based on documentation https://github.com/jameswenzel/mydy/blob/master/src/FileIO.py

test.resolution=480 #qui sto settando quanti tick ho in una quarter note. Quindi ogni quarter note avra 480 ticks.
print(test) #seems that changing the BPM doesn't influence the ticks.
#Resolution is the same as PPQ

mydy.Pattern(format=0, resolution=480, tracks=\
[mydy.Track(relative: True\
  [mydy.TrackNameEvent(tick=0.0, text='', data=[]),
   mydy.InstrumentNameEvent(tick=0.0, text='', data=[]),
   mydy.TimeSignatureEvent(tick=0.0, data=[4, 2, 24, 8]),
   mydy.KeySignatureEvent(tick=0.0, data=[0, 0]),
   mydy.SmpteOffsetEvent(tick=0.0, data=[32, 0, 0, 0, 0]),
   mydy.SetTempoEvent(tick=0.0, data=[9, 163, 27]),
   mydy.SetTempoEvent(tick=0.0, data=[10, 15, 141]),
   mydy.TextMetaEvent(tick=0.0, text='', data=[]),
   mydy.NoteOnEvent(tick=1920.0, channel=9, data=[37, 84]),
   mydy.NoteOnEvent(tick=175.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=305.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=178.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=302.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=205.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=275.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=165.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tic

## Reading notes

The following cell access the loaded MIDI file and reads the  ```Track```. Using the  ```mydy``` library, MIDI files are structured in the following way:

 ```Pattern -> Track -> MIDI_Events ```
 
 This means that, whenever I open a MIDI file, i will get a  ```Pattern ```, which contains  ```Track ```, which contains ```MIDI_Events ```.
 
 This scructure is used because a single MIDI file can contain multiple instruments (guitar, drums, bass ecc...), so each  ```Track ``` corresponds to an instrument, and the  ```MIDI_Events ``` are the note played by the single instrument.
 
 In our case we can assume to be working with single  ```Track ``` MIDI files (we're interested only in drums).
 
 **NB**: i'm using  ```track.make_ticks_abs()``` to convert the time (expressed in ticks/PPQ) from a relative value into an absolute one.<br>The standard representation of MIDI files represents note based on the time elapsed by the previous note. With this command i'm converting the time from relative to absolute (each note is represented with the time elapsed by the beginning of the song) 

In [29]:

track = test[0] #selecting the track (since it's only one, it will be always at index 0)
track_abs = track.make_ticks_abs()# Converting time from relative to an absolute measure


filtered_list=track_abs.filter(lambda e: isinstance(e, Events.NoteOnEvent))# Selects only Note_On events, i'm discarding the note off

#Just a bunch of test printings 
print(filtered_list)
print(len(filtered_list))
print(filtered_list[2].data[1])
print(test.resolution)


mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=1920.0, channel=9, data=[37, 84]),
   mydy.NoteOnEvent(tick=2095.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=2400.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=2578.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=2880.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=3085.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=3360.0, channel=9, data=[37, 64]),
   mydy.NoteOnEvent(tick=3525.0, channel=9, data=[37, 0]),
   mydy.NoteOnEvent(tick=3840.0, channel=9, data=[36, 113]),
   mydy.NoteOnEvent(tick=3840.0, channel=9, data=[49, 111]),
   mydy.NoteOnEvent(tick=4320.0, channel=9, data=[42, 108]),
   mydy.NoteOnEvent(tick=4800.0, channel=9, data=[42, 108]),
   mydy.NoteOnEvent(tick=5040.0, channel=9, data=[36, 113]),
   mydy.NoteOnEvent(tick=5040.0, channel=9, data=[57, 103]),
   mydy.NoteOnEvent(tick=5280.0, channel=9, data=[36, 113]),
   mydy.NoteOnEvent(tick=5280.0, channel=9, data=[49, 111]),
   mydy.

## Creating couples (Pitch, time)
So basically now i'm processing the previous raw data (as you can see from the printings, it's quite a mess) in order to obtain couples of ```(pitch, time)``` for each note.


In [30]:
notes = []
couple = ()

for element in filtered_list:
    
    ### test printings
    print(element.tick)
    print(element.data[0])
    ###
    
    notes.append(tuple((element.data[0],element.tick))) #storing couples of (note_pitch,tick_time)


print('printing tuples')
print(notes)


1920.0
37
2095.0
37
2400.0
37
2578.0
37
2880.0
37
3085.0
37
3360.0
37
3525.0
37
3840.0
36
3840.0
49
4320.0
42
4800.0
42
5040.0
36
5040.0
57
5280.0
36
5280.0
49
5520.0
36
5520.0
57
5760.0
36
5760.0
49
6240.0
42
6720.0
42
7200.0
36
7200.0
49
7680.0
36
7680.0
49
8160.0
42
8640.0
42
8880.0
36
8880.0
57
9120.0
36
9120.0
49
9360.0
36
9360.0
57
9600.0
36
9600.0
49
9840.0
47
9960.0
47
10080.0
47
10200.0
47
10320.0
47
10440.0
47
10560.0
47
10800.0
43
10920.0
43
11040.0
43
11160.0
43
11280.0
38
11400.0
38
11520.0
36
11520.0
49
12000.0
42
12480.0
42
12720.0
36
12720.0
57
12960.0
36
12960.0
49
13200.0
36
13200.0
57
13440.0
36
13440.0
49
13665.0
47
13680.0
47
14145.0
47
14160.0
47
14640.0
38
15105.0
47
15120.0
47
15360.0
36
15360.0
49
15840.0
42
16320.0
42
16560.0
36
16560.0
57
16800.0
36
16800.0
49
17040.0
36
17040.0
57
17280.0
36
17280.0
49
19440.0
38
19680.0
38
19800.0
38
19920.0
38
20040.0
38
20160.0
36
20160.0
49
20400.0
42
20640.0
38
20640.0
42
20880.0
42
21120.0
36
21120.0
42
21360.0
42
2160

190560.0
42
190800.0
42
191040.0
36
191040.0
42
191280.0
42
191520.0
38
191520.0
42
191760.0
42
192000.0
36
192000.0
42
192240.0
42
192480.0
38
192480.0
42
192720.0
42
192840.0
42
192960.0
36
192960.0
42
193200.0
42
193440.0
38
193440.0
49
193680.0
42
193920.0
36
193920.0
42
194160.0
42
194400.0
38
194400.0
42
194640.0
42
194880.0
36
194880.0
42
195120.0
42
195360.0
38
195360.0
42
195600.0
42
195840.0
36
195840.0
42
196080.0
38
196080.0
57
196320.0
36
196560.0
38
196560.0
49
196800.0
36
196800.0
42
197040.0
42
197280.0
38
197280.0
49
197520.0
42
197760.0
36
197760.0
42
198000.0
42
198240.0
38
198240.0
42
198480.0
42
198720.0
36
198720.0
42
198960.0
42
199200.0
38
199200.0
42
199440.0
42
199680.0
36
199680.0
42
199920.0
42
200160.0
38
200160.0
42
200400.0
42
200520.0
42
200640.0
36
201120.0
38
201120.0
49
201600.0
36
201720.0
38
201840.0
38
201960.0
38
202080.0
43
202320.0
38
202440.0
38
202560.0
36
202560.0
49
202800.0
36
203040.0
38
203120.0
38
203200.0
47
203280.0
43
203360.0
43
2034

In [31]:
#RANDOM TEST ON THE LIBRARY, USELESS
test= Constants.C_3
print(test)

36


## Trouble and make it double
Now we're going into the tough part.

The following cell is an *utility* in order to convert our MIDI into a text file (that we will use to traing our network, read the reference model here: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/)

Basically it creates 2 classes: ```Note``` and ```Note_List```.

**1)** *Note*:    Creates a simple ```Note``` object composed by ```pitch``` ```c_tick``` and ```idx```. ```pitch``` and ```c_tick``` are the already mentioned pitch and time, while ```idx``` is a variable that counts the index of my note on a 16-th note reference. <br>**For example**: if i have 2 whole notes, the second note will have ```idx=16```.

**2)** *Note_list*:  Creates an empty list where we will add our *Note* objects. It also contains supports methods used to create and manage this list.

**List of methods**: I'll add some comments to the code and make them more readable.


In [67]:
from mydy import Events, FileIO, Containers
import pdb

#Function responsible for converting midi notes into text. Since i have to train my network over the structure i decided
#which is 0b0000000 for no note, 0b01000000 for kick ecc... i need to convert midi notes into this format.

#The original script used for midi-text translation has been lost, must be re-implemented again
PPQ = 480 # Pulse per quater note. Used in sequencers. Standard value
event_per_bar = 16 # to quantise.
min_ppq = PPQ / (event_per_bar/4)

# ignore: 39 hand clap, 54 tambourine, 56 Cowbell, 58 Vibraslap, 60-81

#the dictionary below maps values to other ones. Reduced the size of the used notes. For example
#if i have an eletric snare or a stick snare, i just map both of them into a standard snare

drum_conversion = {35:36, # acoustic bass drum -> bass drum (36)
					37:38, 40:38, # 37:side stick, 38: acou snare, 40: electric snare
					43:41, # 41 low floor tom, 43 ghigh floor tom
					47:45, # 45 low tom, 47 low-mid tom
					50:48, # 50 high tom, 48 hi mid tom
					44:42, # 42 closed HH, 44 pedal HH
					57:49, # 57 Crash 2, 49 Crash 1
					59:51, 53:51, 55:51, # 59 Ride 2, 51 Ride 1, 53 Ride bell, 55 Splash
					52:49 # 52: China cymbal
					}

#Used in the code to map elements, everything that has not one of the following number is discarded.
#Basically i'm ignoring notes that are not in my dataset (for examle i'll ignore shakers ecc...)
				# k, sn,cHH,oHH,LFtom,ltm,htm,Rde,Crash
allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49] # 46: open HH
cymbals_pitch = [49, 51] # crash, ride
cymbals_pitch = [] # crash, ride
# pitch_to_midipitch = {36:midi.C_2, # kick # for general MIDI Drum map
# 						38:midi.D_2, # Snare
# 						39:midi.Eb_2, # hand clap (it's alive by mistake..)
# 						41:midi.F_2, # Low floor tom
# 						42:midi.Gb_2, # Close HH
# 						45:midi.A_2, # Low tom
# 						46:midi.Bb_2, # Open HH
# 						48:midi.C_3,  # Hi Mid Tom
# 						49:midi.Db_3, # Crash
# 						51:midi.Eb_3 # Ride
# 						}

#mapping midi values into notes
pitch_to_midipitch = {36:Constants.C_3,  # for logic 'SoCal' drum mapping
						38:Constants.D_3, 
						39:Constants.Eb_3,
						41:Constants.F_3,
						42:Constants.Gb_3,
						45:Constants.A_3,
						46:Constants.Bb_3,
						48:Constants.C_4,
						49:Constants.Db_4,
						51:Constants.Eb_4
						}
#la singola nota è un elemento composto da pitch (numerico, pitch midi) e tick (modo per tenere il tempo in midi)
class Note:
	def __init__(self, pitch, c_tick):
		self.pitch = pitch
		self.c_tick = c_tick # cumulated_tick of a midi note

	def add_index(self, idx):
		'''index --> 16-th note-based index starts from 0'''
		self.idx = idx

class Note_List():
	def __init__(self):
		''''''
		self.notes = []
		self.quantised = False
		self.max_idx = None

	def add_note(self, note):
		'''note: instance of Note class'''
		self.notes.append(note)

	def quantise(self, minimum_ppq):
		'''
		e.g. if minimum_ppq=120, quantise by 16-th note.
		
		'''
		if not self.quantised:
			for note in self.notes:
				note.c_tick = ((note.c_tick+minimum_ppq/2)//minimum_ppq)* minimum_ppq # quantise
				#here the index is calculated. The index is an absolute index over the 16th notes.
				#for example an index of value 34, means that my current note appears after 34 chromes
				#it's simply calculated by dividing the cumulated tick of the note by the ticks contained in a 16th note
				note.add_index(note.c_tick/minimum_ppq)
            #NB: THE QUANTIZATION FUNCTION ITERATES OVER ALL THE NOTES. So first i add all the notes, then i iterate and quantize

			#Does this automatically reference to the last item added?
			#YES. The counter note will store the last element of the iteration. So basically here i'm assigning as max index the index of the last added note
			self.max_idx = note.idx

			#Here checks if if my ending is a full musical bar. For example, if my file ends with a single kick, i'll add that note.
			#but that kick will (probably) be at the beginning of the last musical bar. So i have to "pad" until the end.
			#It's like adding a pause on my piece, so i have all complete bars and no trucated ones at the end
			if (self.max_idx + 1) % event_per_bar != 0:
				self.max_idx += event_per_bar - ((self.max_idx + 1) % event_per_bar) # make sure it has a FULL bar at the end.
			self.quantised = True

		return

	def simplify_drums(self):
		''' use only allowed pitch - and converted not allowed pitch to the similar in a sense of drums!
		'''
        #Here forces conversion into the pitches in drum_conversion
		for note in self.notes:
			if note.pitch in drum_conversion: # ignore those not included in the key
				note.pitch = drum_conversion[note.pitch]
		#https://stackoverflow.com/questions/30670310/what-do-brackets-in-a-for-loop-in-python-mean
		#The following one is a list comprehension. Basically generates a new list from an existing one using a given condition on the elements
		self.notes = [note for note in self.notes if note.pitch in allowed_pitch]	
				
		return
	
	def return_as_text(self):
		''''''
		length = int(self.max_idx + 1) # of events in the track.
		#print(type(length))
		event_track = []
		#Thw following cycle create a 9 by N matrix. I append N times a vector of nine zeros.
		#This means that i create N notes, and then i initialize them with all zeros (9 zeros, since a note is represented by a 9 element binary number)

		for note_idx in range(length):  #sostituire xrange con range in Python3
			event_track.append(['0']*len(allowed_pitch))
			
		num_bars = length/event_per_bar# + ceil(len(event_texts_temp) % _event_per_bar)

		for note in self.notes:
			pitch_here = note.pitch
			#The following line returns the index of the passed pitch. Basically given an input generic pitch
			#it returns the associated pitch in my vocabolary (computes the actual mapping from the whole
			#vocabolary of notes into my reduced one)
			note_add_pitch_index = allowed_pitch.index(pitch_here) # 0-8
			#print(type(note.idx))  
			#print(type(note_add_pitch_index))
			event_track[int(note.idx)][note_add_pitch_index] = '1'
			# print note.idx, note.c_tick, note_add_pitch_index, ''.join(event_track[note.idx])
			# pdb.set_trace()
			
		event_text_temp = ['0b'+''.join(e) for e in event_track] # encoding to binary
		
		event_text = []
		# event_text.append('SONG_BEGIN')
		# event_text.append('BAR')
		print(num_bars)
		print(type(num_bars))        
		for bar_idx in range(int(num_bars)):
			event_from = bar_idx * event_per_bar
			event_to = event_from + event_per_bar
			event_text = event_text + event_text_temp[event_from:event_to]
			event_text.append('BAR')

		# event_text.append('SONG_END')

		return ' '.join(event_text)
		

## Creating Note_List
As simple as that, i'm taking my starting list of couples ```(pitch,time)``` into ```Note```  objects. 

Why? Because you should recycle also code, not only plastic. LUL



In [68]:

##NB: AGGIUNGERE GLI idx ALLE SINGLE NOTES! 
note_list = Note_List()
for note in notes:
    pitch = note[0]
    tick = note[1]
    idx = int(tick / min_ppq)
    new_note = Note(pitch,tick)
    new_note.add_index(idx)
    note_list.add_note(new_note)


In [69]:
for note in note_list.notes:
    print(note.idx)

16
17
20
21
24
25
28
29
32
32
36
40
42
42
44
44
46
46
48
48
52
56
60
60
64
64
68
72
74
74
76
76
78
78
80
80
82
83
84
85
86
87
88
90
91
92
93
94
95
96
96
100
104
106
106
108
108
110
110
112
112
113
114
117
118
122
125
126
128
128
132
136
138
138
140
140
142
142
144
144
162
164
165
166
167
168
168
170
172
172
174
176
176
178
180
180
182
184
184
186
188
188
190
192
192
194
196
196
198
200
200
202
204
204
206
208
208
210
212
212
214
216
216
218
220
220
222
224
224
226
228
228
230
232
232
233
234
235
236
236
238
238
240
240
242
244
244
246
248
248
250
252
252
254
256
256
258
260
260
262
264
264
266
268
268
270
272
272
274
276
276
278
280
280
282
284
284
286
286
288
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
325
326
327
328
328
330
332
332
334
336
336
338
340
340
342
344
344
346
348
348
350
352
352
354
355
356
356
358
358
360
360
364
364
366
368
368
370
372
372
374
376
376
378
380
380
382
384
384
386
388
388
390
391
392
392
394
396
396
398
400
400
402
404
404

In [70]:

print('printing ticks before quantization')
for note in note_list.notes:
    print(note.c_tick)
    
note_list.quantise(min_ppq)

print('printing ticks after quantization')
for note in note_list.notes:
    print(note.c_tick)

printing ticks before quantization
1920.0
2095.0
2400.0
2578.0
2880.0
3085.0
3360.0
3525.0
3840.0
3840.0
4320.0
4800.0
5040.0
5040.0
5280.0
5280.0
5520.0
5520.0
5760.0
5760.0
6240.0
6720.0
7200.0
7200.0
7680.0
7680.0
8160.0
8640.0
8880.0
8880.0
9120.0
9120.0
9360.0
9360.0
9600.0
9600.0
9840.0
9960.0
10080.0
10200.0
10320.0
10440.0
10560.0
10800.0
10920.0
11040.0
11160.0
11280.0
11400.0
11520.0
11520.0
12000.0
12480.0
12720.0
12720.0
12960.0
12960.0
13200.0
13200.0
13440.0
13440.0
13665.0
13680.0
14145.0
14160.0
14640.0
15105.0
15120.0
15360.0
15360.0
15840.0
16320.0
16560.0
16560.0
16800.0
16800.0
17040.0
17040.0
17280.0
17280.0
19440.0
19680.0
19800.0
19920.0
20040.0
20160.0
20160.0
20400.0
20640.0
20640.0
20880.0
21120.0
21120.0
21360.0
21600.0
21600.0
21840.0
22080.0
22080.0
22320.0
22560.0
22560.0
22800.0
23040.0
23040.0
23280.0
23520.0
23520.0
23760.0
24000.0
24000.0
24240.0
24480.0
24480.0
24720.0
24960.0
24960.0
25200.0
25440.0
25440.0
25680.0
25920.0
25920.0
26160.0
26400.0
264

141120.0
141360.0
141600.0
141600.0
141840.0
142080.0
142080.0
142320.0
142560.0
142560.0
142800.0
143040.0
143040.0
143280.0
143520.0
143520.0
143760.0
144000.0
144000.0
144240.0
144480.0
144480.0
144720.0
144960.0
144960.0
145200.0
145440.0
145440.0
145680.0
145920.0
145920.0
146160.0
146400.0
146400.0
146640.0
146880.0
146880.0
147120.0
147240.0
147360.0
147480.0
147600.0
147720.0
147840.0
147840.0
148080.0
148320.0
148320.0
148560.0
148800.0
148800.0
149040.0
149280.0
149280.0
149520.0
149760.0
149760.0
150000.0
150000.0
150120.0
150120.0
150240.0
150240.0
150480.0
150480.0
150600.0
150600.0
150720.0
150720.0
150960.0
150960.0
151080.0
151080.0
151200.0
151200.0
151440.0
151440.0
151680.0
151680.0
152160.0
152160.0
152640.0
152640.0
152880.0
153120.0
153120.0
153600.0
153600.0
153840.0
154080.0
154080.0
154560.0
154800.0
154800.0
155040.0
155280.0
155280.0
155520.0
155520.0
155760.0
156000.0
156000.0
156240.0
156480.0
156480.0
156720.0
156960.0
156960.0
157200.0
157440.0
157440.0
1

In [71]:
note_list.simplify_drums()

In [72]:
txt = note_list.return_as_text()

121.0
<class 'float'>


In [73]:
print(txt)

0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 BAR 0b010000000 0b010000000 0b000000000 0b000000000 0b010000000 0b010000000 0b000000000 0b000000000 0b010000000 0b000000000 0b010000000 0b000000000 0b010000000 0b010000000 0b000000000 0b000000000 BAR 0b100000001 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b100000001 0b000000000 0b100000001 0b000000000 0b100000001 0b000000000 BAR 0b100000001 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b100000001 0b000000000 0b000000000 0b000000000 BAR 0b100000001 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b100000001 0b000000000 0b100000001 0b000000000 0b100000001 0b000000000 BAR 0b100000001 0b000000